In [1]:
!pip install crewai duckduckgo_search huggingface_hub python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.1/614.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB

In [7]:
!rm -rf /content/multi-agent-research-lab
!git clone https://github.com/verachamochumbi/multi-agent-research-lab.git

Cloning into 'multi-agent-research-lab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 138 (delta 52), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (138/138), 64.99 KiB | 5.00 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [11]:
import os
os.environ["OPENAI_API_KEY"] = "fake-key-not-used"


In [13]:
%%writefile /content/multi-agent-research-lab/src/crewai_workflow.py
from crewai import Agent, Task, Crew, Process

from .agents import ejecutar_flujo_simple


def crear_agentes_crewai(tema: str):
    """Crea agentes CrewAI descriptivos (no usan LLM, solo estructura)."""

    investigador_agent = Agent(
        role="Investigador",
        goal=f"Buscar información relevante sobre: {tema}",
        backstory=(
            "Investigador que sabe encontrar artículos y recursos fiables "
            "sobre IA y salud utilizando herramientas de búsqueda en la web."
        ),
        llm=None,  # No usamos LLM de CrewAI
    )

    redactor_agent = Agent(
        role="Redactor científico",
        goal=(
            "Redactar un informe claro y estructurado en formato Markdown "
            "basado en las notas del investigador."
        ),
        backstory=(
            "Redactor especializado en comunicar resultados de investigación "
            "de forma comprensible y ordenada."
        ),
        llm=None,  # No usamos LLM de CrewAI
    )

    revisor_agent = Agent(
        role="Revisor",
        goal=(
            "Revisar el informe, evaluar claridad y coherencia, "
            "y proponer mejoras."
        ),
        backstory=(
            "Revisor crítico que detecta problemas de claridad "
            "y propone ajustes para mejorar el texto."
        ),
        llm=None,  # No usamos LLM de CrewAI
    )

    return investigador_agent, redactor_agent, revisor_agent


def crear_tareas_crewai(investigador_agent, redactor_agent, revisor_agent, tema: str):
    """Crea las tareas CrewAI que describen el flujo de trabajo."""

    tarea_investigacion = Task(
        description=(
            f"Investigar el tema '{tema}' buscando fuentes en la web y "
            "recopilando fragmentos de texto relevantes."
        ),
        expected_output="Notas y fragmentos de texto sobre el tema.",
        agent=investigador_agent,
    )

    tarea_redaccion = Task(
        description=(
            "A partir de las notas del investigador, redactar un informe de ~500 palabras "
            "en formato Markdown con las secciones: Introducción, Hallazgos clave, "
            "Desafíos éticos y técnicos y Conclusión."
        ),
        expected_output="Informe en Markdown estructurado.",
        agent=redactor_agent,
        context=[tarea_investigacion],
    )

    tarea_revision = Task(
        description=(
            "Revisar el informe redactado y añadir un comentario con feedback sobre "
            "claridad, coherencia y posibles mejoras."
        ),
        expected_output="Comentario de revisión y texto mejorado.",
        agent=revisor_agent,
        context=[tarea_redaccion],
    )

    return tarea_investigacion, tarea_redaccion, tarea_revision


def crear_crew(tema: str) -> Crew:
    """Construye el Crew con 3 agentes y 3 tareas."""

    investigador_agent, redactor_agent, revisor_agent = crear_agentes_crewai(tema)
    tarea_investigacion, tarea_redaccion, tarea_revision = crear_tareas_crewai(
        investigador_agent, redactor_agent, revisor_agent, tema
    )

    crew = Crew(
        agents=[investigador_agent, redactor_agent, revisor_agent],
        tasks=[tarea_investigacion, tarea_redaccion, tarea_revision],
        process=Process.sequential,
        # Ojo: NO usamos verbose numérico ni booleano para evitar problemas de versión
    )

    return crew


def ejecutar_flujo_crewai(tema: str) -> str:
    """
    Ejecuta el flujo de colaboración multiagente.

    CrewAI se usa para definir:
      - Agentes (Investigador, Redactor, Revisor)
      - Tareas y proceso secuencial

    La ejecución real (búsqueda → resumen → revisión) se delega a
    ejecutar_flujo_simple() definido en agents.py, que ya está probado.
    """

    # 1) Creamos el crew (estructura multiagente requerida por la rúbrica)
    crew = crear_crew(tema)
    print("\n[INFO] CrewAI configurado con 3 agentes y 3 tareas.\n")

    # 2) Llamamos a la lógica que ya funciona (investigar → redactar → revisar)
    print("[FLUJO REAL] Ejecutando flujo simple (investigar → redactar → revisar)...")
    resultado = ejecutar_flujo_simple(tema)

    return resultado


Overwriting /content/multi-agent-research-lab/src/crewai_workflow.py


In [14]:
import sys, os, importlib

repo_path = "/content/multi-agent-research-lab"
if repo_path not in sys.path:
    sys.path.append(repo_path)

import src.crewai_workflow
importlib.reload(src.crewai_workflow)

from src.crewai_workflow import ejecutar_flujo_crewai


In [15]:
tema = "Impacto de los datos sintéticos en la atención médica"
resultado = ejecutar_flujo_crewai(tema)
print(resultado)


[INFO] CrewAI configurado con 3 agentes y 3 tareas.

[FLUJO REAL] Ejecutando flujo simple (investigar → redactar → revisar)...
[Agente Investigador] Buscando en la web sobre: Impacto de los datos sintéticos en la atención médica impacto datos sintéticos en salud


/content/multi-agent-research-lab/src/tools.py:17: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


[Agente Escritor] Generando resumen con Hugging Face...
[Agente Revisor] Revisando el borrador...

Comentario del revisor: El texto presenta una estructura clara (introducción, hallazgos, desafíos y conclusión). Podría reforzarse el tono académico con algunas transiciones más formales y, si es posible, incluir uno o dos ejemplos concretos de uso de datos sintéticos en entornos clínicos reales.

# Introducción

En este informe se analiza el impacto del uso de datos sintéticos en el ámbito de la atención médica. A partir de distintas fuentes en línea se resumen las oportunidades, riesgos y desafíos asociados al empleo de datos generados artificialmente para entrenar y evaluar modelos de inteligencia artificial aplicados a la salud.


## Hallazgos clave

- Los datos sintéticos permiten ampliar conjuntos de datos reales, lo que puede mejorar el rendimiento de modelos de IA cuando los datos clínicos son escasos o sensibles.
- Facilitan el intercambio de información al reducir el riesgo de r

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [18]:
with open("research_summary.md", "w", encoding="utf-8") as f:
    f.write(resultado)

print("Archivo 'research_summary.md' guardado correctamente!")



Archivo 'research_summary.md' guardado correctamente!


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [19]:
import os
os.listdir()


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

['.config', 'multi-agent-research-lab', 'research_summary.md', 'sample_data']

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [20]:
from google.colab import files
files.download("research_summary.md")


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

<IPython.core.display.Javascript object>

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

<IPython.core.display.Javascript object>

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag